In [1]:
import ollama
import pandas as pd
import evaluate
from scipy.stats import ks_2samp
from io import StringIO
import json

C:\Users\Manuel\Desktop\Cosecose\PR1\TestBiometria\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Config

In [2]:
MODEL_NAME = "qwen2.5:3b"

# Dataset di domande e risposte
with open('qa_dataset.json', 'r', encoding='utf-8') as f:
    qa_dataset = json.load(f)

# Dati reali per confronto tabellare
real_data = pd.read_excel("2022.11ÎË▒Ý▓╣│õÍð╬─Ê¹╩│/Shanghai_T1DM_Summary.xlsx")

## Funzioni per evaluation del qa

In [3]:
def query_ollama(prompt):
    """Manda un prompt al modello Ollama e restituisce la risposta."""
    response = ollama.chat(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}]
    )
    return response['message']['content']

def evaluate_qa(predictions, references):
    """Valuta domande e risposte con BERTScore."""
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, lang="it")
    return results

def run_qa_evaluation():
    print("\n=== Valutazione Risposte a Domande ===")
    predictions = []
    references = []

    for item in qa_dataset:
        prompt = f"Rispondi in modo conciso alla domanda: {item['question']}"
        response = query_ollama(prompt)
        predictions.append(response.strip())
        references.append(item['answer'])

    results = evaluate_qa(predictions, references)
    f1_scores = results['f1']
    f1_medio = sum(f1_scores) / len(f1_scores)

    print("\nBERTScore medio (F1):", round(f1_medio, 4))

## Funzioni per tabular evaluation (TDB)

In [ ]:
import pandas as pd
import re
from scipy.stats import ks_2samp
from io import StringIO


def fetch_synthetic_data(prompt):
    """Invia una query per ottenere i dati sintetici sotto forma di tabella."""
    response = query_ollama(prompt)
    return response


def extract_table_from_response(response):
    """Estrae la tabella da una risposta testuale."""
    match = re.search(r"(\|.*\|)", response, re.DOTALL)
    if match:
        return match.group(0)
    return None


def load_data_from_string(table_data):
    """Carica i dati da una stringa in un DataFrame Pandas."""
    df = pd.read_csv(StringIO(table_data), sep='|', engine='python', skipinitialspace=True)

    # Rimuove la colonna "Unnamed: 0" che potrebbe essere stata generata
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    # Normalizza i nomi delle colonne rimuovendo spazi extra e uniformando a minuscolo
    df.columns = df.columns.str.strip().str.lower()
    return df


def ks_test_between_columns(real_column, synthetic_column):
    """Esegue il test di Kolmogorov-Smirnov tra due colonne."""
    try:
        statistic, p_value = ks_2samp(real_column, synthetic_column)
        return {"KS-statistic": statistic, "p-value": p_value}
    except Exception as e:
        return {"error": str(e)}


def evaluate_synthetic_data(real_data, synthetic_data):
    """Valuta la qualità dei dati sintetici confrontandoli con i dati reali."""
    ks_results = {}
    for column in real_data.columns:
        # Controlla se la colonna esiste anche nei dati sintetici
        if column in synthetic_data.columns:
            ks_results[column] = ks_test_between_columns(real_data[column], synthetic_data[column])
        else:
            ks_results[column] = {"error": f"Colonna '{column}' mancante nei dati sintetici."}
    return ks_results


def print_ks_results(ks_results):
    """Stampa i risultati del test KS per ogni colonna."""
    for column, result in ks_results.items():
        if "error" in result:
            print(f"\nColonna '{column}': Errore - {result['error']}")
        else:
            print(f"\nColonna '{column}': KS-statistic={result['KS-statistic']:.4f}, p-value={result['p-value']:.4f}")


def run_tabular_evaluation(real_data):
    print("\n=== Generazione e Valutazione Dati Tabulari ===")

    prompt = """
    Genera una tabella con le seguenti indicazioni, prima dei : avrai l'header della colonna e dopo avrai che tipo di valore mettere all'interno della cella.

    La tabella deve contenere circa 20 righe di dati realistici che siano *simili* ai seguenti valori (ma *non identici*):
    - Patient Number: numeri sequenziali univoci.
    - High-Density Lipoprotein Cholesterol: valori tipici tra 0.70 e 2.50 mmol/L.
    - Gender: 1 (Femmina) o 2 (Maschio).
    - Low-Density Lipoprotein Cholesterol: valori tra 1.50 e 4.50 mmol/L.
    - Age: età in anni, tipicamente tra 30 e 80.
    - Creatinine: livelli tra 40 e 120 umol/L.
    - Height: valori tra 1.50 e 1.90 m.
    - Estimated Glomerular Filtration Rate: valori tra 50 e 160 ml/min/1.73m2.
    - Weight: tra 50 e 90 kg.
    - Uric Acid: valori tra 150 e 420 mmol/L.
    - BMI: calcolato in kg/m2, tra 18.5 e 30.
    - Blood Urea Nitrogen: livelli tra 2.5 e 7.0 mmol/L.
    - Smoking History: anni o pacchetti di storia da 0 a 50.
    - Hypoglycemia: “yes” o “no”.
    - Alcohol Drinking History: “drinker” o “non-drinker”.
    - Type of Diabetes: tipicamente "T1DM" (Type 1 Diabetes Mellitus).
    - Duration of Diabetes: durata in anni, tra 0 e 30.
    - Acute Diabetic Complications: descrizioni brevi o "none".
    - Diabetic Macrovascular/Microvascular Complications: brevi elenchi o "none".
    - Comorbidities: elenchi brevi o "none".
    - Hypoglycemic Agents/Other Agents: descrizioni brevi di farmaci.
    - Parametri di glucosio, insulina e colesterolo: valori tipici coerenti con i livelli realistici.

    Genera ogni riga con dati casuali ma realistici, mantenendo coerenza tra i valori correlati (ad esempio, BMI deve essere congruente con altezza e peso). Se non hai bisogno di dettagli per alcune colonne, puoi inserire “/” o lasciare il campo vuoto.

    RISPONDI SOLO CON LA TABELLA. NON SCRIVERE ALTRO.
    """

    try:
        # Ottieni i dati sintetici
        response = fetch_synthetic_data(prompt)

        # Estrai la tabella
        table_data = extract_table_from_response(response)
        if not table_data:
            raise ValueError("Nessuna tabella trovata nella risposta.")

        # Carica i dati sintetici in un DataFrame
        synthetic_data = load_data_from_string(table_data)

        print("Tabella sintetica:\n", synthetic_data)

        # Valuta la qualità dei dati sintetici confrontandoli con quelli reali
        ks_results = evaluate_synthetic_data(real_data, synthetic_data)

        # Stampa i risultati
        print_ks_results(ks_results)

    except Exception as e:
        print(f"Errore: {e}")


In [4]:
run_qa_evaluation()


=== Valutazione Risposte a Domande ===

BERTScore medio (F1): 0.7301


In [ ]:
run_tabular_evaluation(real_data)